## Script KPI (squelette ETL)

#### Import de la librairie (Pandas)

In [2]:
import pandas as pd

In [3]:
# Charger dataset
df = pd.read_csv("maintenance_events.csv", parse_dates=["failure_date"])

# Tri par machine + date
df = df.sort_values(by=["machine", "failure_date"])

# Initialisation résultats
results = []

machines = df["machine"].unique()

for machine in machines:
    data = df[df["machine"] == machine].copy()
    
    # MTTR = moyenne des temps de réparation
    mttr = data["repair_time_h"].mean()
    
    # MTBF = moyenne du temps entre 2 pannes (jours)
    if len(data) > 1:
        diffs = data["failure_date"].diff().dt.days[1:]
        mtbf = diffs.mean()
    else:
        mtbf = None  # Pas assez de données
    
    # Downtime total
    downtime_total = data["downtime_h"].sum()
    
    # Coût total
    cost_total = data["cost_eur"].sum()
    
    # Scrap total & scrap rate (par rapport au total scrap global)
    scrap_total = data["scrap_units"].sum()
    scrap_rate = scrap_total / df["scrap_units"].sum()
    
    # OEE simplifié (proxy) = MTBF / (MTBF + MTTR)
    if mtbf and mttr:
        oee = mtbf / (mtbf + mttr)
    else:
        oee = None
    
    results.append({
        "machine": machine,
        "MTTR_h": round(mttr, 2),
        "MTBF_days": round(mtbf, 2) if mtbf else None,
        "Downtime_total_h": downtime_total,
        "Cost_total_eur": cost_total,
        "Scrap_total": scrap_total,
        "Scrap_rate": round(scrap_rate*100, 2),
        "OEE_proxy": round(oee*100, 2) if oee else None
    })

# Résumé
summary = pd.DataFrame(results)


#### Sauvegarde CSV

In [4]:
summary.to_csv("kpis_summary.csv", index=False)

print("✅ KPIs calculés et enregistrés dans kpis_summary.csv")
print(summary)

✅ KPIs calculés et enregistrés dans kpis_summary.csv
   machine  MTTR_h  MTBF_days  Downtime_total_h  Cost_total_eur  Scrap_total  \
0  Press_1    4.00      13.08                52            3475          355   
1  Press_2    4.64      12.85                65            4450          314   
2  Press_3    3.73      11.36                56            3875          333   
3  Press_4    4.00      12.54                56            4300          395   
4  Press_5    3.60      11.57                54            4550          269   

   Scrap_rate  OEE_proxy  
0       21.31      76.59  
1       18.85      73.45  
2       19.99      75.26  
3       23.71      75.81  
4       16.15      76.27  


Nous obtenons un fichier kpis_summary.csv avec des colonnes :

machine

MTTR_h

MTBF_days

Downtime_total_h

Cost_total_eur

Scrap_total

Scrap_rate (%)

OEE_proxy (%)